Сверточные нейронные сети извлекают данные из изображение по кускам. Чем больше будет инпут лэйеров, тем более тщательно будет происходить процедура считывания. 

1) Поступают данные на вход


2) Процесс "свертывания" - грубо говоря здесь происходит создание признаков


3) Объединение


3.5) Каждый раз, когда нейросеть свертывает и объединяет данные, то создается hidden layer (скрытый слой)


4) Один раз создается Fully Connected Layer (Полностью связаный слой), который по сути также является скрытым слоем и представляет из себя обычный слой нейронной сети


5) Выходные данные

Поговорим о том, что такое "свертывание". Свертывание делит изображение, которое поступает на вход в сеть. Изображение делится на несколько окон. К примеру, 5 на 5. Внутри этих пяти оконо мы берем только 3-на-3 окон пространство, которое называется "клетка" В этом пространстве модель ищет признаки, которые помогут классифицировать изображение или дать окну "название". Далее мы перемещаем клетку, к примеру на 2 окна и у нас появляется новая клетка. Таким образом мы можем получить 4 клетки.

Перейдем к "объединению". Мы получили 4 клетки. Каждая клетка может иметь какое-то значение внутри. К примеру мы получили такое же пространство из клеток 5-на-5, в каждом единичное окне лежат данные, которые мы получили на выходе из свертывания. Процесс объединение происходит точно также, как и свертывание, но разница в том, что при свертывании мы создавала признаки, а при объединении мы упрощаем выходные данные из свертывания. В матрице 5-на-5 мы снова берем клетку 3-на-3 единичных окон. Ищем самое большое значение внутри 3-на-3 клетки, среди единичных клеток и таким образом мы получаем новую клетку. Так мы получим снова 4 клетки, если будем перемещать на 2 единичных клетки. 

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, LSTM
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#physd = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physd[0], True)


X = pickle.load(open('X.pickle', 'rb'))
y = pickle.load(open('y.pickle', 'rb'))

Num GPUs Available:  0


In [2]:
#не дает модели использовать более 0.333 оперативки, помогает если надо обучать несколько моделей сразу (к примеру, когда
#мы учили несколько моделей сразу играть в игру)
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.99)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [3]:
NAME = 'Cats-vs-dog-cnn-64x2-{}'.format(int(time.time())) #изменим имя модели

Далее нужно нормализировать данные перед тем как они пройдут через нейронную сеть:

In [4]:
X = X/255.0

In [5]:
X.shape[1:] #70 листов по 70 элементов в каждом - размер изображения, 1 - цвета

(50, 50, 1)

Анализ модели

In [6]:
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
#После вызовам модели с этим параметром у нас появится папка в папке, где хранится проект

In [9]:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=X.shape[1:])) #добавляем слой сверточной сети 
# (3,3) - размер окна, input_shape - сообщаем о форме входных данных, мы могли бы просто написать (50, 50, 1)
#Conv всегда идет с Pooling, но можно после Conv сделать и активацию
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2))) #pool_size - размер окна, состоящего из 4 клеток, которые мы получили на выходе из
#свертывания

model.add(Conv2D(64, (3,3))) #добавляем слой сверточной сети
#Conv всегда идет с Pooling, но можно после Conv сделать и активацию
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Теперь нужно сделать данные плоскими, т.к. сверточная нейронная сеть на выходе должна получать плоские данные (в 2D)
model.add(Flatten())

model.add(Dense(64)) #Мы можем и не добавлять этот слой, но для более хорошей работы лучше добавим
model.add(Activation('relu'))

model.add(Dense(1)) #На выходе должен быть 1D-array
model.add(Activation('sigmoid'))

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

model.compile(loss='binary_crossentropy', #т.к. у нас только два класса 
             optimizer='adam', #всегда используем его
             metrics=['accuracy']) 

model.fit(X, y, batch_size=32, epochs=10, validation_split=0.3, callbacks=[tensorboard])
#callbacks - это лист в котором находятся моменты обучения модели

Epoch 1/10
546/546 [==============================] - 11s 19ms/step - loss: 0.6229 - accuracy: 0.6474 - val_loss: 0.5940 - val_accuracy: 0.6754
Epoch 2/10
546/546 [==============================] - 10s 19ms/step - loss: 0.5203 - accuracy: 0.7436 - val_loss: 0.4982 - val_accuracy: 0.7592
Epoch 3/10
546/546 [==============================] - 10s 19ms/step - loss: 0.4667 - accuracy: 0.7786 - val_loss: 0.5188 - val_accuracy: 0.7547
Epoch 4/10
546/546 [==============================] - 10s 19ms/step - loss: 0.4244 - accuracy: 0.8093 - val_loss: 0.4663 - val_accuracy: 0.7790
Epoch 5/10
546/546 [==============================] - 10s 19ms/step - loss: 0.3761 - accuracy: 0.8298 - val_loss: 0.4606 - val_accuracy: 0.7896
Epoch 6/10
546/546 [==============================] - 10s 19ms/step - loss: 0.3286 - accuracy: 0.8570 - val_loss: 0.4651 - val_accuracy: 0.7827
Epoch 7/10
546/546 [==============================] - 10s 19ms/step - loss: 0.2800 - accuracy: 0.8805 - val_loss: 0.4819 - val_accuracy:

# Анализ модели

Либо ты можешь зайти anaconda3, там зайти в environment, там запустить tensorflow с помощью terminal дальше переходить в свою папку:

cd C:\Users\yanka\Project\\"Depp Learning Basics with Tensowr Flow and Keras"

Дальше вводишь

tensorboard --logdir logs

Далее появится ссылка на дашборд, вставляешь её в браузер и все

Для удобства просмотра используем \w в поиске

Здесь нужно в основном смотреть на то, как ведет себя val_loss. Даже если val_accuracy увеличивается, а val_loss возрастает снова в какой-то точке, то если мы добавим больше эпох, то в опр. момент acc начнет уменьшаться.

In [8]:
#Здесь мы будем перебирать параметры. Лучше начинать с самых основных. Находим параметры, которые реально учат модель и от них
#отталкиваемся. 

dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            #имя модели с параметрами и время её создания (чтобы запомнить)
            MODEL_NAME = '{} - conv, {} - nodes, {} - dense - {}'.format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir='/logs'.format(MODEL_NAME))
            print()
            print(MODEL_NAME)
            print()
            
            model = Sequential()
            model.add(Conv2D(layer_size, (3,3), input_shape=X.shape[1:])) #добавляем слой сверточной сети,
            #первый слой должен иметь input_shape
            # (3,3) - размер окна (не знаю что это, но вроде kernel size) 
            #Conv всегда идет с Pooling, но можно после Conv сделать и активацию
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1): #-1, потому что у нас уже есть один conv_layer сверху на инпуте. Добавим переменную conv
                model.add(Conv2D(layer_size, (3,3))) #добавляем слой сверточной сети, добавляем переменную в слой
                # (3,3) - размер окна (не знаю что это)
                #Conv всегда идет с Pooling, но можно после Conv сделать и активацию
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2,2)))
                
            #Теперь нужно сделать данные плоскими, т.к. сверточная нейронная сеть на выходе должна получать 
            #перый Dense должен иметь перед собой Flatten
            model.add(Flatten())
            
            for l in range(dense_layer):
                model.add(Dense(layer_size)) #помещаем переменную из цикла для перебора 
                model.add(Activation('relu'))

            model.add(Dense(1)) #На выходе должен быть 1D-array, видимо

            model.add(Activation('sigmoid'))
            
            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy', #т.к. у нас только два класса 
                         optimizer='adam', #всегда используем его
                         metrics=['accuracy']) 

            model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks =[tensorboard])
            #callbacks - это лист в котором находятся моменты обучения модели


1 - conv, 32 - nodes, 0 - dense - 1659028751

Epoch 1/10
702/702 [==============================] - 6s 8ms/step - loss: 0.6184 - accuracy: 0.6602 - val_loss: 0.5811 - val_accuracy: 0.7038
Epoch 2/10
702/702 [==============================] - 6s 8ms/step - loss: 0.5499 - accuracy: 0.7249 - val_loss: 0.5579 - val_accuracy: 0.7218
Epoch 3/10
702/702 [==============================] - 6s 8ms/step - loss: 0.5194 - accuracy: 0.7496 - val_loss: 0.5545 - val_accuracy: 0.7114
Epoch 4/10
702/702 [==============================] - 6s 8ms/step - loss: 0.4970 - accuracy: 0.7630 - val_loss: 0.5367 - val_accuracy: 0.7347
Epoch 5/10
702/702 [==============================] - 6s 8ms/step - loss: 0.4812 - accuracy: 0.7765 - val_loss: 0.5270 - val_accuracy: 0.7447
Epoch 6/10
702/702 [==============================] - 6s 8ms/step - loss: 0.4726 - accuracy: 0.7780 - val_loss: 0.5279 - val_accuracy: 0.7403
Epoch 7/10
702/702 [==============================] - 6s 8ms/step - loss: 0.4564 - accuracy: 0.7897 -

KeyboardInterrupt: 

**Для этой модели logs хранятся в папке C:\**

Дальше мы смотрим на графики. Из графика мы поняли, что с задачей лучше справляются модели с 0 dense_layers и 3 conv_layers. Поэтому, мы обновляем листы и оставляем 0 dense_layer, и 3 conv, а также делаем 64 нейрона (просто так). И убираем в цикле dense_layer переменную и вместо нее вставляем 512 (в данном случае так будет лучше).

In [8]:
#Здесь мы будем перебирать параметры. Лучше начинать с самых основных. Находим параметры, которые реально учат модель и от них
#отталкиваемся. 

dense_layers = [0]
layer_sizes = [64]
conv_layers = [2]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            #имя модели с параметрами и время её создания (чтобы запомнить)
            MODEL_NAME = '{} - conv, {} - nodes, {} - dense - {}'.format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir='/logs'.format(MODEL_NAME))
            print(MODEL_NAME)
            print()
            
            model = Sequential()
            model.add(Conv2D(layer_size, (3,3), input_shape=X.shape[1:])) #добавляем слой сверточной сети,
            #первый слой должен иметь input_shape
            # (3,3) - размер окна (не знаю что это, но вроде kernel size) 
            #Conv всегда идет с Pooling, но можно после Conv сделать и активацию
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1): #-1, потому что у нас уже есть один conv_layer сверху на инпуте. Добавим переменную conv
                model.add(Conv2D(layer_size, (3,3))) #добавляем слой сверточной сети, добавляем переменную в слой
                # (3,3) - размер окна (не знаю что это)
                #Conv всегда идет с Pooling, но можно после Conv сделать и активацию
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2,2)))
                
            #Теперь нужно сделать данные плоскими, т.к. сверточная нейронная сеть на выходе должна получать 
            #перый Dense должен иметь перед собой Flatten
            model.add(Flatten())
            
            for l in range(dense_layer):
                model.add(Dense(layer_size)) #помещаем переменную из цикла для перебора 
                model.add(Activation('relu'))

            model.add(Dense(1)) #На выходе должен быть 1D-array, видимо

            model.add(Activation('sigmoid'))
            
            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy', #т.к. у нас только два класса 
                         optimizer='adam', #всегда используем его
                         metrics=['accuracy']) 

            model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks =[tensorboard])
            #callbacks - это лист в котором находятся моменты обучения модели
            
model.save('64x3-CNN.model') #сохраняем модель (сохраняем весы и байес)         

2 - conv, 64 - nodes, 0 - dense - 1659023576

Epoch 1/10
702/702 [==============================] - 13s 18ms/step - loss: 0.6070 - accuracy: 0.6635 - val_loss: 0.5355 - val_accuracy: 0.7255
Epoch 2/10
702/702 [==============================] - 11s 16ms/step - loss: 0.5159 - accuracy: 0.7497 - val_loss: 0.5003 - val_accuracy: 0.7475
Epoch 3/10
702/702 [==============================] - 17s 24ms/step - loss: 0.4766 - accuracy: 0.7755 - val_loss: 0.5141 - val_accuracy: 0.7379
Epoch 4/10
702/702 [==============================] - 11s 16ms/step - loss: 0.4477 - accuracy: 0.7929 - val_loss: 0.4671 - val_accuracy: 0.7756
Epoch 5/10
702/702 [==============================] - 11s 15ms/step - loss: 0.4250 - accuracy: 0.8064 - val_loss: 0.4562 - val_accuracy: 0.7892
Epoch 6/10
702/702 [==============================] - 11s 15ms/step - loss: 0.4045 - accuracy: 0.8201 - val_loss: 0.4554 - val_accuracy: 0.7820
Epoch 7/10
702/702 [==============================] - 11s 15ms/step - loss: 0.3843 - accur

INFO:tensorflow:Assets written to: 64x3-CNN.model\assets


INFO:tensorflow:Assets written to: 64x3-CNN.model\assets


Даннаяя модель начинает переучиваться, если дать ей dense_layer. Это из-за того что у нас задача бинарной классификации, однако, если бы у нас была мильтиклассификация к примеру из датасета MNIST_Fashion, то нам нужна была бы более сложная модель

Чтобы загрузить модель мы можем использовать 

Сначала обработаем изображение так, как был обработан остальной датасет

In [ ]:
def prepare(filepath):
    IMG_SIZE = 70
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
model = tf.keras.models.load_model('64x3-CNN.model') #загружаем модель

prediction = model.predict([prepare('dog.jpg')]) #делаем предсказание, обработав сэмпл

print(CATEGORIES[int(prediction[0][0])])

